In [1]:
## 📚 1. Setup and Data Preparation (The Full Workflow)

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit # Essential for correct CV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# --- 1.1. Best Parameters from Optuna (Notebook 04) ---
# We hard-code the optimal parameters found in the previous tuning notebook for the final model build.
BEST_PARAMS = {
    'n_layers': 1,
    'n_units': 39,
    'learning_rate': 0.006207,
    'batch_size': 32
}

# --- 1.2. Data Loading, Engineering, and Scaling (Reproduction) ---
file_path = '../../datasets/Supplement_Sales_Weekly_Expanded.csv'
try:
    data = pd.read_csv(file_path)
except:
    raise FileNotFoundError("Please ensure the Supplement_Sales_Weekly_Expanded.csv file path is correct.")

# Feature Engineering (replicate steps from Notebook 04)
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data = data.drop(columns=['Category', 'Revenue', 'Location'], errors='ignore')

# Select and process one product's data
product_data_grouped = data.groupby(['Product_Name', 'Year', 'Month']).agg(
    Price_Avg=('Price', 'mean')
).reset_index()
product_data_grouped = product_data_grouped.sort_values(by=['Product_Name', 'Year', 'Month']).reset_index(drop=True)
PRODUCT_ID = product_data_grouped['Product_Name'].unique()[0]
product_data = product_data_grouped[product_data_grouped['Product_Name'] == PRODUCT_ID].copy()

# Add Time-Series Features (Lags and Moving Average)
product_data['Time_Index'] = np.arange(len(product_data)) + 1
product_data['Time_Index_Squared'] = product_data['Time_Index'] ** 2
product_data['Price_Lag_1'] = product_data['Price_Avg'].shift(1)
product_data['Price_Lag_3'] = product_data['Price_Avg'].shift(3)
product_data['Price_MA_6'] = product_data['Price_Avg'].rolling(window=6).mean().shift(1)
product_data = product_data.dropna().reset_index(drop=True)

FEATURES = ['Year', 'Month', 'Time_Index', 'Time_Index_Squared', 
            'Price_Lag_1', 'Price_Lag_3', 'Price_MA_6']
TARGET = 'Price_Avg'

X = product_data[FEATURES].values
y = product_data[TARGET].values

# Scaling
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

print(f"Data prepared for final evaluation: {len(X)} samples.")

Data prepared for final evaluation: 57 samples.


In [2]:
## ⚙️ 2. Building the Final Tuned Model

def build_tuned_model(params, input_dim):
    model = Sequential()
    
    # Input Layer and First Hidden Layer
    model.add(Dense(params['n_units'], activation='relu', input_shape=(input_dim,)))
    
    # Additional Hidden Layers (based on best n_layers)
    for _ in range(params['n_layers'] - 1): # If n_layers is 1, this loop is skipped
        model.add(Dense(params['n_units'], activation='relu'))
        
    # Output Layer for Regression
    model.add(Dense(1))

    # Compile the model with the best learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])
    return model

tuned_nn_model = build_tuned_model(BEST_PARAMS, X_scaled.shape[1])
tuned_nn_model.summary()

c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 39)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            40 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 352 (1.38 KB)

 Trainable params: 352 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
## 📊 3. Combined Cross-Validation and Evaluation

# Define the TimeSeriesSplit (using 5 splits for better reliability, versus 3 in the quick tuning phase)
tscv = TimeSeriesSplit(n_splits=5) 

scaled_mae_scores = []
mae_actual_scores = []
r2_scores = []
fold_count = 1

print(f"\nStarting Final TimeSeries Cross-Validation ({tscv.n_splits} folds)...")

for train_index, test_index in tscv.split(X_scaled):
    print(f"\n--- Fold {fold_count} ---")
    
    # 1. Split Data
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train_scaled, y_test_scaled = y_scaled[train_index], y_scaled[test_index]
    y_test_actual = y[test_index] # Keep the actual values for unscaled evaluation

    # 2. Rebuild and Train Model (Crucial for CV)
    # The model must be reset/rebuilt for each fold to prevent information leakage!
    model_fold = build_tuned_model(BEST_PARAMS, X_scaled.shape[1])
    
    model_fold.fit(
        X_train, y_train_scaled,
        epochs=100, # Use more epochs now that parameters are finalized
        batch_size=BEST_PARAMS['batch_size'],
        verbose=0,
        shuffle=False
    )
    
    # 3. Predict and Inverse Transform
    y_pred_scaled = model_fold.predict(X_test, verbose=0)
    y_pred_actual = scaler_y.inverse_transform(y_pred_scaled)

    # 4. Evaluate Scores (Scaled and Actual)
    scaled_mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
    actual_mae = mean_absolute_error(y_test_actual, y_pred_actual)
    r2 = r2_score(y_test_actual, y_pred_actual)

    scaled_mae_scores.append(scaled_mae)
    mae_actual_scores.append(actual_mae)
    r2_scores.append(r2)
    
    print(f"  Scaled MAE: {scaled_mae:.4f}")
    print(f"  Actual MAE: ${actual_mae:.2f}")
    print(f"  R-squared (R2): {r2:.4f}")
    
    fold_count += 1

# --- 4. Final Summary ---
print("\n==================================")
print(f"FINAL AVERAGE SCALED MAE: {np.mean(scaled_mae_scores):.4f}")
print(f"FINAL AVERAGE ACTUAL MAE: ${np.mean(mae_actual_scores):.2f}")
print(f"FINAL AVERAGE R2 SCORE: {np.mean(r2_scores):.4f}")
print("==================================")


Starting Final TimeSeries Cross-Validation (5 folds)...

--- Fold 1 ---
  Scaled MAE: 1.2028
  Actual MAE: $7.76
  R-squared (R2): -1.3220

--- Fold 2 ---


c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Scaled MAE: 0.5380
  Actual MAE: $3.47
  R-squared (R2): 0.0919

--- Fold 3 ---


c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Scaled MAE: 3.0380
  Actual MAE: $19.60
  R-squared (R2): -11.7745

--- Fold 4 ---


c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Scaled MAE: 0.8630
  Actual MAE: $5.57
  R-squared (R2): -0.2893

--- Fold 5 ---


c:\f5\CrossValidationAndHyperparameterTuningTraining\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Scaled MAE: 0.8527
  Actual MAE: $5.50
  R-squared (R2): -0.0278

FINAL AVERAGE SCALED MAE: 1.2989
FINAL AVERAGE ACTUAL MAE: $8.38
FINAL AVERAGE R2 SCORE: -2.6643


## **📉 Analysis of Final Cross-Validation Results**

**The goal of using TimeSeriesSplit was to get an honest, reliable assessment. The results are indeed honest, revealing a model with poor generalization ability across the different prediction periods (folds).**

### **1\. The Core Metrics**

| Metric | Value | Interpretation |
| :---- | :---- | :---- |
| **Final Avg. Actual MAE** | **$8.38** | **On average, the model's price prediction is off by $8.38. Given the typical price range of a product, this is likely a large error, indicating low practical accuracy.** |
| **Final Avg. $\\text{R}^2$ Score** | **\-2.6643** | **This is the most telling result. A negative $\\text{R}^2$ score means the model performs worse than simply predicting the mean (average) of the price in the test set. In practical terms, it means the model is failing to capture the underlying price patterns.** |
| **Final Avg. Scaled MAE** | **1.2989** | **An average error of 1.3 standard deviations is quite high for a model, confirming the poor performance observed in the $\\text{R}^2$.** |

### **2\. Variability Across Folds (The Problem)**

**Notice the massive swing in performance between the folds:**

| Fold | Scaled MAE | Actual MAE | R2 Score | Interpretation |
| :---- | :---- | :---- | :---- | :---- |
| **2** | **0.5380** | **$3.47** | **0.0919** | **Best performance, showing it *can* sometimes capture the pattern.** |
| **3** | **3.0380** | **\*\*$19.60\*\*** | **\-11.7745** | **Worst performance. The model failed completely on this period, likely due to an extreme price change or unexpected market event.** |

**This huge variability confirms that the model is brittle; it works somewhat in certain time periods but collapses in others, indicating it hasn't learned robust, generalizable features.**

### **3\. Conclusion: Is This the Best Model?**

**No, this is definitively not the best model for this dataset.**

**While the *hyperparameters* found by Optuna might be optimal for this specific Neural Network architecture, the $\\text{R}^2$ score tells us that the overall modeling approach (simple feed-forward NN \+ current features) is inadequate for the complexity and non-linearity of the price changes.**

## 🌟 4. Conclusion and Final Rationale

### The Integrated Workflow

This final notebook demonstrates the **integrated and robust** workflow required for any mission-critical prediction task:

1.  **Feature Engineering:** Creating relevant time-series features (lags, MAs).
2.  **Hyperparameter Tuning:** Using advanced methods (Optuna) to find the best model structure.
3.  **Correct Cross-Validation:** Using **TimeSeriesSplit** to simulate real-world sequential forecasting.

### 🛑 Analysis of Final Performance (The Reality Check)

The rigorous TimeSeries Cross-Validation provided a crucial reality check:

| Metric | Result | Meaning |
| :--- | :--- | :--- |
| **Final Avg. Actual MAE** | **\$8.38** | The average price prediction error is too high for reliable forecasting. |
| **Final Avg. R² Score** | **-2.6643** | The model performs **worse than simply guessing the average price**. This indicates a failure to capture the underlying patterns, despite hyperparameter tuning. |

The high variability in scores across the folds (MAE from \$3.47 to \$19.60) shows the model is **brittle** and cannot generalize robustly across different market periods.

### 🎯 Why This Dataset is Ideal for Combined CV and Tuning

Your **`Supplement_Sales_Weekly_Expanded.csv`** dataset perfectly highlights the need for this combined approach, even when the final result is poor:

* **Temporal Dependency (Time-Series):** This necessitated **TimeSeriesSplit**. Without it, we would have seen an inflated $\text{R}^2$ score (likely close to 1.0) and falsely believed the model was excellent.
* **Complex Modeling Requirement:** The dynamic, non-linear price changes demanded a powerful tuner (Optuna) and model (Neural Network).

**Conclusion on the Model:**

While the hyperparameter tuning successfully found the *best possible structure* for this specific **Feed-Forward Neural Network**, the final low $\text{R}^2$ score proves that the **model architecture itself, or the features provided, is insufficient** to reliably forecast the price variability of this product.

**The next logical steps, informed by this robust CV process, would be to:**
1.  **Engineer richer features** (e.g., incorporate sales data, external economic indicators).
2.  **Use a more appropriate time-series model** (e.g., an **RNN/LSTM** or a **Tree-based model** like XGBoost, which often handle time-series complexity better than simple NNs).